In [ ]:
! git clone https://github.com/KTH-HPC/sputniPIC-DD2360.git

Cloning into 'sputniPIC-DD2360'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 78 (delta 0), reused 6 (delta 0), pack-reused 68
Receiving objects: 100% (78/78), 52.13 KiB | 10.43 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
%cd /content/sputniPIC-DD2360/
! make
%cd /content/sputniPIC-DD2360/bin
! mkdir -p data
!./sputniPIC.out /content/sputniPIC-DD2360/inputfiles/GEM_2D.inp

/content/sputniPIC-DD2360
nvcc -I./include -arch=sm_75 -std=c++11 -O3 -g -Xcompiler -Wall --compiler-bindir=g++ src/particles.cu -c -o src/particles.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/ConfigFile.cpp -c -o src/ConfigFile.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/sputniPIC.cpp -c -o src/sputniPIC.o
In file included from src/sputniPIC.cpp:25:
./include/IC.h: In function ‘void initGEM(parameters*, grid*, EMfield*, EMfield_aux*, particles*, interpDensSpecies*)’:
./include/IC.h:82:25: warning: unused variable ‘sign’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
   82 |     double prob, theta, sign;
      |                         ^~~~
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/EMfield.cpp -c -o src/EMfield.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/InterpDensNet.cpp -c

In [ ]:
def cf():

    files = ["rhoe_10.vtk", "rho_net_10.vtk", "rhoi_10.vtk"]
    EPSILON = 0.015

    def read_file(path):
        with open(path, "r") as file:
            return file.read().split()

    for file in files:
        path_gpu = f"/content/sputniPIC-DD2360/bin/data_gpu/{file}"
        path_cpu = f"/content/sputniPIC-DD2360/bin/data/{file}"

        gpu_data = read_file(path_gpu)
        cpu_data = read_file(path_cpu)

        if len(cpu_data) != len(gpu_data):
            print("Error: Different lengths")
            return 0

        for i in range(len(gpu_data)):
            try:
                f_gpu = float(gpu_data[i])
                f_cpu = float(cpu_data[i])

                diff = abs(f_gpu - f_cpu)

                if diff >= EPSILON:
                    print("Error: Large Difference")
                    return 0
            except ValueError:
                if gpu_data[i] != cpu_data[i]:
                    print("Error: Not Equal", gpu_data[i], cpu_data[i])
                    return 0

    return 1

cf()